# Text search

![Vespa logo](https://vespa.ai/assets/vespa-logo-color.png)

This self-contained tutorial will create a basic text search application based on the MS MARCO dataset,
similar to Vespa's [text search tutorials](https://docs.vespa.ai/en/tutorials/text-search.html).

[Install pyvespa](https://pyvespa.readthedocs.io/) and start Docker, validate minimum 4G available:

In [ ]:
!docker info | grep "Total Memory"

## Create an application package

Create an [application package](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.ApplicationPackage) - do not use a `-` in the name:

In [ ]:
from vespa.package import ApplicationPackage

app_package = ApplicationPackage(name="textsearch")

## Add fields to the schema

Add [fields](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.Field)
to the application's [schema](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.Schema):

In [ ]:
from vespa.package import Field

app_package.schema.add_fields(
    Field(name = "id",    type = "string", indexing = ["attribute", "summary"]),
    Field(name = "title", type = "string", indexing = ["index", "summary"], index = "enable-bm25"),
    Field(name = "body",  type = "string", indexing = ["index", "summary"], index = "enable-bm25")
)

* `id` holds the document ids, while `title` and `body` are the text fields of the documents.

* Setting `"index"` in `indexing` means that a searchable index for `title` and `body` is created.
  Read more about [indexing options](https://docs.vespa.ai/en/reference/schema-reference.html#indexing). 

* Setting `index = "enable-bm25"` will pre-compute quantities to make it fast to compute the BM25 score.

## Search multiple fields when querying

A [FieldSet](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.FieldSet)
groups fields together for searching -
it configures queries to look for matches both in the titles and bodies of the documents:

In [ ]:
from vespa.package import FieldSet

app_package.schema.add_field_set(
    FieldSet(name = "default", fields = ["title", "body"])
)

## Define how to rank the documents matched

Specify how to rank the matched documents by defining a
[RankProfile](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.RankProfile).
Here, the `bm25` rank profile combines BM25 scores from `title` and `body`:

In [ ]:
from vespa.package import RankProfile

app_package.schema.add_rank_profile(
    RankProfile(name = "default", first_phase = "bm25(title) + bm25(body)")
)
app_package.schema.add_rank_profile(
    RankProfile(name = "bm25", first_phase = "bm25(title) + bm25(body)")
)
app_package.schema.add_rank_profile(
    RankProfile(name = "native_rank", first_phase = "nativeRank(title, body)")
)

## Deploy

The text search app with fields, a fieldset to group fields together, and a rank profile to rank matched documents is now defined and ready to deploy.
Deploy `app_package` on the local machine using Docker,
without leaving the notebook, by creating an instance of
[VespaDocker](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.deployment.VespaDocker):

In [ ]:
import os
from vespa.deployment import VespaDocker

vespa_docker = VespaDocker()
app = vespa_docker.deploy(application_package=app_package)

`app` now holds a [Vespa](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.application.Vespa) instance,
to be used to interact with the application.
`pyvespa` provides an API to define Vespa application packages from python.
`vespa_docker.deploy` exports Vespa configuration files to `disk_folder` -
going through these is a good way to learning about Vespa configuration.

## Feed

Download approx 10K documents:

In [ ]:
from pandas import read_csv

docs = read_csv(
    filepath_or_buffer="https://data.vespa.oath.cloud/blog/msmarco/sample_docs.csv"
)
docs.head()

Feed the DataFrame to the application:

In [ ]:
feed_res = app.feed_df(docs)

## Query

Query the text search app using the [Vespa Query language](https://docs.vespa.ai/en/query-language.html)
by sending the parameters to the body argument of
[app.query](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.application.Vespa.query):

In [ ]:
query = {
    'yql': 'select * from sources * where userQuery()',
    'query': 'what keeps planes in the air',
    'ranking': 'bm25',
    'type': 'all',
    'hits': 10
}

In [ ]:
res = app.query(body=query)
res.hits[0]

## Query with QueryModel

Using the Vespa Query Language as above gives full query power and flexibility from Vespa.
In contrast, the QueryModel abstraction found in
[learntorank](https://vespa-engine.github.io/learntorank/notebooks/query-model.html)
focuses on specific use cases and can be more useful for ML experiments.

## Cleanup

In [ ]:
vespa_docker.container.stop()
vespa_docker.container.remove()